In [ ]:
pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: gast
    Found existing installation: gast 0.6.0
    Uninstalling gast-0.6.0:
      

In [ ]:
# LOAD DATA & DOWNLOAD DATASET
import tensorflow as tf
import pandas as pd
from tensorflow.keras.datasets import boston_housing
(train_x, train_y), (test_x, test_y) = boston_housing.load_data()
features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


In [ ]:
#CONVERT TO PANDAS DATASET
df_train_x = pd.DataFrame(train_x, columns=features)
df_test_x=pd.DataFrame(test_x, columns=features)
df_train_y = pd.DataFrame(train_y, columns=['MEDV'])
df_test_y = pd.DataFrame(test_y, columns=['MEDV'])
df_train_x.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [ ]:
#Feature Normalization
#Normalize the data using the mean and std of the training data.
#First calculate the mean and std using training data.Then normalize the training and test data using the calculated mean and std.
#The test data should always be normalized using training data and std.
mean = df_train_x.mean(axis=0)
std = df_train_x.std(axis=0)
df_train_x -=mean
df_test_x -= mean
df_test_x/= std
df_train_x /= std


In [ ]:
#FEATURE COLUMN
feature_columns_numeric=[]
feature_columns_numeric = [tf.feature_column.numeric_column(fname,dtype=tf.float32) for fname in features]

In [ ]:
#Build Data Pipeline
def estimator_input_fn(df_data, df_label, epochs=10, shuffle=True, batch_size=32):
  def input_funct():
    ds=tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
    if shuffle:
      ds=ds.shuffle(100)
    ds=ds.batch(batch_size).repeat(epochs)
    return ds
  return input_funct #Return the inner function

train_input_fn=estimator_input_fn(df_train_x, df_train_y)
#The df_test_y dataframe should be used here instead of df_train_y
val_input_fn=estimator_input_fn(df_test_x, df_test_y, epochs=1, shuffle=False)

#Build Model
model=tf.estimator.LinearRegressor(feature_columns=feature_columns_numeric,optimizer='RMSProp')

result=model.train(train_input_fn, steps=1000)

res=model.evaluate(val_input_fn)
print(res)

{'average_loss': 32.510662, 'label/mean': 23.078432, 'loss': 44.6426, 'prediction/mean': 22.784998, 'global_step': 130}


In [ ]:
#Predict Result
result1=model.predict(val_input_fn)
for pred ,exp in zip(result1, test_y[:32]):
  print('Predicted Value: ', pred['predictions'][0], 'Expected: ', exp)


Predicted Value:  11.254671 Expected:  7.2
Predicted Value:  21.155388 Expected:  18.8
Predicted Value:  19.330145 Expected:  19.0
Predicted Value:  36.179775 Expected:  27.0
Predicted Value:  24.358994 Expected:  22.2
Predicted Value:  20.482332 Expected:  24.5
Predicted Value:  26.34578 Expected:  31.2
Predicted Value:  22.99259 Expected:  22.9
Predicted Value:  18.64544 Expected:  20.5
Predicted Value:  23.746243 Expected:  23.2
Predicted Value:  20.280186 Expected:  18.6
Predicted Value:  17.719166 Expected:  14.5
Predicted Value:  17.454325 Expected:  17.8
Predicted Value:  33.838272 Expected:  50.0
Predicted Value:  20.599226 Expected:  20.8
Predicted Value:  17.37751 Expected:  24.3
Predicted Value:  23.055054 Expected:  24.2
Predicted Value:  20.68807 Expected:  19.8
Predicted Value:  20.916786 Expected:  19.1
Predicted Value:  27.678055 Expected:  22.7
Predicted Value:  13.313742 Expected:  12.0
Predicted Value:  17.928629 Expected:  10.2
Predicted Value:  21.214176 Expected: 